In [3]:
#导入必要的库
import numpy as np 
import pandas as pd
from collections import  Counter
import matplotlib.pyplot as plt  
from sklearn.datasets import  load_iris

#实现计算熵的函数
def calculate_entropy(y):
    """计算数据集的熵"""
    if len(y) ==0:
        return 0
    #统计每个类别的数量
    counter=Counter(y)  
    
    """计算熵"""
    entropy=0
    for count in counter.values():
        probablity=count/len(y)
        entropy-=probablity*np.log2(probablity)
        
    return entropy


"""实现计算信息增益的函数"""
def calculate_information_gain(X,y,feature_index):
    """计算按照feature_index划分数据集后的信息增益"""
    
    #计算父节点的熵
    parent_entropy=calculate_entropy(y)
    
    #获取该特征的所有唯一值
    feature_values=np.unique(X[:,feature_index])
    
    #计算按照该特征划分后的熵
    children_entropy=0
    for value in feature_values:
        #获取该特征值对应的索引
        child_indexs=np.where(X[:,feature_index]==value)[0]
        
        #计算子节点的权重
        weight=len(child_indexs)/len(y)
        #计算子节点的熵
        child_entropy=calculate_entropy(y[child_indexs])
        #累加子节点的熵
        children_entropy+=weight*child_entropy
        
    #计算信息增益
    information_gain=parent_entropy-children_entropy
    
    return information_gain

"""实现决策树的节点类"""
class DecisionNode:  
    """决策树的节点类"""
    def __init__(self,feature_index=None,threshold=None,left=None,right=None,value=None):
        """初始化节点"""
        self.feature_index=feature_index #特征索引
        self.threshold=threshold #特征阈值
        self.left=left  #左子树
        self.right=right #右子树
        self.value=value #节点的预测值 
        
"""实现决策树的树类"""
class DecisionTree: 
    """自己实现的决策树"""
    def __init__(self,max_depth=None):
        self.max_depth=max_depth  #最大深度
        self.root=None
    def _best_spilt(self,X,y):
        """寻找最佳的划分特征以及划分的阈值"""
        m,n=X.shape
        if m<=1 :
            return None,None
        
        #计算父节点的熵 
        parent_entropy=calculate_entropy(y)
        
        #计算最佳信息增益和对应的特征 
        best_gain=0 
        best_feature_index=None
        
        #遍历所有特征
        for feature_index in range(n):
            #计算该信息的信息增益 
            info_gain=calculate_information_gain(X,y,feature_index)
            
            #更新最佳信息增益和对应的特征
            if  info_gain>best_gain:
                best_gain=info_gain
                best_feature_index=feature_index
        return best_feature_index, best_gain        
        
    def _build_tree(self,X,y,depth=0):
        """递归构造决策树"""
        
        m,n=X.shape
        
        #检查停止条件
        if (self.max_depth is not None and depth>=self.max_depth) or len(set(y))==1:
            #如果达到最大深度或者所有样本属于同一类别，则创建叶节点
            leaf_value=Counter(y).most_common(1)[0][0]
            return DecisionNode(value=leaf_value)
        #寻找最佳划分特征和信息增益
        best_feature_index,best_gain=self._best_spilt(X,y)
        
        #如果没找到好的划分特征 
        if best_feature_index is None:
            leaf_value=Counter(y).most_common(1)[0][0] 
            return DecisionNode(value=leaf_value)
        
        #如果找到了最优的划分特征 则按照该划分的特征进行分类工作 
        feature_values=np.unique(X[:,best_feature_index])
        
        #如果特征只有一个值，则创建叶节点 
        if len(feature_values)==1:
            leaf_value=Counter(y).most_common(1)[0][0]
            return DecisionNode(value=leaf_value)
        
        #对于标称型特征，取中位数作为阈值
        threshold=np.median(feature_values)
        #划分数据集
        left_indexs=np.where(X[:,best_feature_index]<threshold)[0]
        right_indexs=np.where(X[:,best_feature_index]>=threshold)[0]
        
        #递归构建左子树和右子树
        left_subtree=self._build_tree(X[left_indexs],y[left_indexs],depth+1)
        right_subtree=self._build_tree(X[right_indexs],y[right_indexs],depth+1)
        
        #返回决策节点
        return DecisionNode(feature_index=best_feature_index, threshold=threshold, left=left_subtree, right=right_subtree)
    
    def fit(self,X,y):
        """训练决策树模型"""
        self.root=self._build_tree(X,y)
        return self    
    
    def predict_sample(self,x,node):
        """预测单个样本"""
        #如果是叶节点，则返回预测值
        if node.value is not None:
            return  node.value  
        
        #如果不是叶节点，则根据特征索引和阈值进行分类 走左子树还是右子树 
        if x[node.feature_index] <node.threshold:
            return self.predict_sample(x, node.left)
        else:
            return self.predict_sample(x,node.right) 
        
    def predict(self,X):
        """预测多个样本"""
        return  np.array([self.predict_sample(x,self.root) for x in X])
    
def visualize_tree(node,feature_names=None,class_names=None,depth=0):
    """可视化决策树"""
    indent=" "* depth
    
    if node.value is not None:
        class_idx=node.value
        class_name=class_names[class_idx] if class_names is not None else str(class_idx)
        print(f"{indent}Leaf: {class_name}")
        return 
    feature_name=feature_names[node.feature_index]if feature_names else f"Feature {node.feature_index}"
    print(f"{indent}{feature_name} < {node.threshold}:")
    
    print(f"{indent}Left:")
    visualize_tree(node.left, feature_names, class_names, depth + 1)
    print(f"{indent}Right:")
    visualize_tree(node.right, feature_names, class_names, depth + 1)
    
#加载数据集进行测试 
iris = load_iris()
X = iris.data
y = iris.target

# 分割训练集和测试集
np.random.seed(42)
indices = np.random.permutation(len(X))
train_size = int(len(X) * 0.8)
X_train, X_test = X[indices[:train_size]], X[indices[train_size:]]
y_train, y_test = y[indices[:train_size]], y[indices[train_size:]]

# 训练决策树模型
tree = DecisionTree(max_depth=3)
tree.fit(X_train, y_train)

# 可视化决策树
print("决策树结构:")
visualize_tree(tree.root, iris.feature_names, iris.target_names)

# 在测试集上评估模型
y_pred = tree.predict(X_test)
accuracy = np.sum(y_pred == y_test) / len(y_test)
print(f"测试集准确率: {accuracy:.4f}")

决策树结构:
petal length (cm) < 4.45:
Left:
 petal length (cm) < 3.3:
 Left:
  sepal width (cm) < 3.5:
  Left:
   Leaf: setosa
  Right:
   Leaf: setosa
 Right:
  Leaf: versicolor
Right:
 petal length (cm) < 5.5:
 Left:
  petal width (cm) < 1.75:
  Left:
   Leaf: versicolor
  Right:
   Leaf: virginica
 Right:
  Leaf: virginica
测试集准确率: 0.9667


# 决策树算法实现与分析实验报告

## 1. 实验概述

本实验实现了一个基于信息增益的决策树分类算法，并在经典的鸢尾花数据集上进行了测试。实验包含了决策树的核心算法实现、模型训练、预测和性能评估等完整流程。

## 2. 理论基础

### 2.1 信息熵（Information Entropy）

信息熵是衡量数据集纯度的重要指标，定义为：

$$H(S) = -\sum_{i=1}^{k} p_i \log_2(p_i)$$

其中：
- $S$ 是数据集
- $k$ 是类别数量
- $p_i$ 是第 $i$ 类样本在数据集中的比例

熵值越小，数据集的纯度越高；熵值为0时，数据集完全纯净（所有样本属于同一类）。

### 2.2 信息增益（Information Gain）

信息增益衡量了按某个特征划分数据集后纯度的提升程度：

$$IG(S, A) = H(S) - \sum_{v \in Values(A)} \frac{|S_v|}{|S|} H(S_v)$$

其中：
- $A$ 是待评估的特征
- $Values(A)$ 是特征 $A$ 的所有可能取值
- $S_v$ 是特征 $A$ 取值为 $v$ 的样本子集

信息增益越大，说明该特征对分类的贡献越大。

### 2.3 决策树构建原理

决策树采用递归的方式构建：
1. 选择信息增益最大的特征作为当前节点的划分标准
2. 根据该特征的取值将数据集分割成子集
3. 对每个子集递归执行上述过程
4. 满足停止条件时创建叶节点

## 3. 算法实现分析

### 3.1 核心函数实现

**熵计算函数**：
- 使用 `Counter` 统计各类别频次
- 计算每个类别的概率
- 应用信息熵公式进行计算

**信息增益计算**：
- 计算父节点熵值
- 遍历特征的所有唯一值
- 计算加权平均的子节点熵值
- 返回信息增益

### 3.2 决策树结构

**节点类设计**：
- `feature_index`：划分特征的索引
- `threshold`：划分阈值
- `left/right`：左右子树
- `value`：叶节点的预测值

**树类实现**：
- `_best_split()`：寻找最佳划分特征
- `_build_tree()`：递归构建决策树
- `predict()`：预测新样本

## 4. 实验结果分析

### 4.1 数据集概况

- **数据集**：鸢尾花数据集（Iris Dataset）
- **样本总数**：150个样本
- **特征维度**：4维（花萼长度、花萼宽度、花瓣长度、花瓣宽度）
- **类别数量**：3类（山鸢尾、变色鸢尾、维吉尼亚鸢尾）
- **训练集**：120个样本（80%）
- **测试集**：30个样本（20%）

### 4.2 模型性能

**测试集准确率**：96.67%

这个结果表明：
- 决策树在鸢尾花数据集上表现优异
- 仅有1个样本分类错误（30个测试样本中）
- 算法实现正确且有效

### 4.3 决策树结构分析

通过可视化输出可以观察到：
- 树的最大深度被限制为3层，防止过拟合
- 每个内部节点选择信息增益最大的特征进行划分
- 叶节点给出最终的分类结果

## 5. 数学原理深度分析

### 5.1 为什么选择信息增益？

信息增益基于信息论，具有以下优势：
- **理论基础扎实**：基于香农信息论
- **直观易懂**：信息增益大意味着分类效果好
- **计算简单**：公式简洁，计算复杂度低

### 5.2 阈值选择策略

对于连续特征，本实现选择中位数作为划分阈值：
- **优点**：简单有效，对异常值不敏感
- **改进空间**：可以尝试所有可能的划分点，选择信息增益最大的

### 5.3 停止条件设计

算法设置了多个停止条件：
- 达到最大深度限制
- 所有样本属于同一类别
- 无法找到有效的划分特征
- 特征只有单一取值

## 6. 算法优缺点分析

### 6.1 优点

- **可解释性强**：决策过程清晰直观
- **无需数据预处理**：能处理数值型和类别型特征
- **计算效率高**：训练和预测速度快
- **处理缺失值能力强**：可以设计相应策略

### 6.2 缺点

- **容易过拟合**：特别是在树深度较大时
- **对噪声敏感**：异常值可能影响划分质量
- **偏向多值特征**：信息增益偏好取值较多的特征
- **不稳定性**：训练数据的微小变化可能导致完全不同的树

## 7. 总结与展望

本实验成功实现了基于信息增益的决策树算法，在鸢尾花数据集上取得了96.67%的高准确率。实验验证了决策树算法的有效性，同时也展示了其良好的可解释性。

**改进方向**：
- 实现信息增益率（C4.5算法）来处理多值特征偏向问题
- 加入剪枝机制来减少过拟合
- 支持连续特征的最优划分点搜索
- 实现随机森林等集成方法提升性能